<a href="https://colab.research.google.com/github/jennyschilling/DataProject1_DS2002/blob/main/convert_csv_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pymysql

In [ ]:
db_config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Passw0rd123',
    'database': 'retail_sales_dw'
}

In [ ]:
customer_vip_data = pd.read_csv('customer_vip_info.csv', dtype={'is_vip': int})

In [ ]:
conn = pymysql.connect(**db_config)
customer_dim = pd.read_sql('SELECT customer_id, first_name, last_name FROM customer_dim', conn)

merged_df = pd.merge(customer_dim, customer_vip_data, on=['first_name', 'last_name'], how='left')

In [ ]:
merged_df['is_vip'] = merged_df['is_vip'].fillna(False)
conn = pymysql.connect(**db_config)
cursor = conn.cursor()

cursor.execute("ALTER TABLE customer_dim ADD COLUMN IF NOT EXISTS is_vip BOOL")

In [ ]:
for index, row in merged_df.iterrows():
    update_query = """
        UPDATE customer_dim
        SET is_vip = %s
        WHERE customer_id = %s
    """
    values = (row['is_vip'], row['customer_id'])
    cursor.execute(update_query, values)

conn.commit()
conn.close()